### Imports and Version Check

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import (ImageDataGenerator, Iterator,
                                       array_to_img, img_to_array, load_img)
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import regularizers
from tensorflow.keras import layers
import sys, os
import matplotlib.pyplot as plt
from sklearn import metrics
import tensorflow_model_optimization as tfmot
import shap
import keras
import matplotlib.cm as cm
from IPython.display import Image
import pandas as pd
import seaborn as sns

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True
# sess = tf.compat.v1.Session(config=config)
# sess.as_default()

# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# for physical_device in physical_devices:
#     tf.config.experimental.set_memory_growth(physical_device, True)

In [3]:
tf.__version__

'2.3.0'

In [4]:
%matplotlib inline

In [5]:
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

### Data Generator

In [6]:
train_data_dir = '../data/CollisionData/'

img_width, img_height = 96, 96 # 224, 224
nb_train_samples = 730 
nb_validation_samples = 181
epochs = 25
batch_size = 16

if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3) 

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    ) # set validation split

#### train with 100% of data. See other notebooks for validation of the models
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary') # set as training data

Found 911 images belonging to 2 classes.


## Model Architectures

In [8]:
models = [] # can be used for multiple models, for this we are using a single model

### Custom CNN

Tiny Compatible

In [9]:
finetune_model_CNN = tf.keras.Sequential([
    layers.InputLayer(input_shape=input_shape),
    layers.BatchNormalization(),
    layers.Conv2D(filters=16, kernel_size=5, padding='same', activation='relu'),
    layers.MaxPool2D(),
    
    layers.BatchNormalization(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPool2D(),
    
    layers.BatchNormalization(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPool2D(),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

finetune_model_CNN._name="Custom_CNN"

finetune_model_CNN.compile(
    optimizer=tf.keras.optimizers.Adam(lr = 1e-5, decay = 1e-5),
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

In [10]:
models.append(finetune_model_CNN)

In [11]:
finetune_model_CNN.summary()

Model: "Custom_CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 96, 96, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 96, 96, 16)        1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        4640      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0

## Training and Testing

In [12]:
callbacks = [
             tf.keras.callbacks.EarlyStopping(
                 monitor='val_loss', patience = 15,
                 min_delta=0.001, restore_best_weights=True
             )
]

history = []

for each in models:
    print("="*40)
    print("Training and Testing Model: %s" % str(each.name))
    temp_history = each.fit(train_generator, 
        steps_per_epoch = nb_train_samples // batch_size, 
        epochs = epochs, 
        validation_steps = nb_validation_samples // batch_size, shuffle=True, callbacks = callbacks) 
    print("="*40)
    history.append(temp_history)


Training and Testing Model: Custom_CNN
Epoch 1/25
45/45 [==============================] - 9s 200ms/step - accuracy: 0.6801 - loss: 0.6365
Epoch 2/25
45/45 [==============================] - 9s 202ms/step - accuracy: 0.7177 - loss: 0.5648
Epoch 3/25
45/45 [==============================] - 9s 202ms/step - accuracy: 0.7719 - loss: 0.4866
Epoch 4/25
45/45 [==============================] - 9s 200ms/step - accuracy: 0.7778 - loss: 0.4656
Epoch 5/25
45/45 [==============================] - 9s 202ms/step - accuracy: 0.7816 - loss: 0.4519
Epoch 6/25
45/45 [==============================] - 9s 201ms/step - accuracy: 0.8095 - loss: 0.4209
Epoch 7/25
45/45 [==============================] - 9s 199ms/step - accuracy: 0.8053 - loss: 0.3922
Epoch 8/25
45/45 [==============================] - 9s 200ms/step - accuracy: 0.8220 - loss: 0.3586
Epoch 9/25
45/45 [==============================] - 9s 201ms/step - accuracy: 0.8414 - loss: 0.3634
Epoch 10/25
45/45 [==============================] - 9s 201ms

### Model Summary

#### Number of Parameters

In [13]:
for each in models:
    print("Number of parameters:  %s" % str(each.name))
    print(each.count_params())

Number of parameters:  Custom_CNN
1258829


#### Number of Ops

In [14]:
def get_flops(model_h5_path):
    session = tf.compat.v1.Session()
    graph = tf.compat.v1.get_default_graph()

    with graph.as_default():
        with session.as_default():
            model = tf.keras.models.load_model(model_h5_path)
            run_meta = tf.compat.v1.RunMetadata()
            opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
            flops = tf.compat.v1.profiler.profile(graph=graph,
                                                  run_meta=run_meta, cmd='op', options=opts)

    tf.compat.v1.reset_default_graph()

    return flops.total_float_ops

for each in models:
    model_file = '../my-log-dir/saved_model/' + str(each.name) + '.h5'
    print("Number of OPS:  %s" % str(each.name))
    print(get_flops(model_file))

Number of OPS:  Custom_CNN
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
2516668


#### Model Size

In [15]:
import os

for each in models:
    model_file = '../my-log-dir/saved_model/' + str(each.name) + '.h5'
    print("Model:  %s" % str(each.name))
    b = os.path.getsize(model_file)
    print ("Size(mb): %d" % (b/1000000))



Model:  Custom_CNN
Size(mb): 15


# Optimization

## Pruning

In [16]:
model_file = '../my-log-dir/saved_model/Custom_CNN.h5'
model = tf.keras.models.load_model(model_file)

In [17]:
end_step = np.ceil(1.0 * nb_train_samples / batch_size).astype(np.int32) * epochs

pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=0,
                                                   end_step=end_step,
                                                   frequency=100)

model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model, pruning_schedule=pruning_schedule)
model_for_pruning.summary()

Instructions for updating:
Please use `layer.add_weight` method instead.
Model: "Custom_CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_batch_no (None, 96, 96, 3)         13        
_________________________________________________________________
prune_low_magnitude_conv2d ( (None, 96, 96, 16)        2418      
_________________________________________________________________
prune_low_magnitude_max_pool (None, 48, 48, 16)        1         
_________________________________________________________________
prune_low_magnitude_batch_no (None, 48, 48, 16)        65        
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 48, 48, 32)        9250      
_________________________________________________________________
prune_low_magnitude_conv2d_2 (None, 48, 48, 32)        18466     
_________________________________________________

In [18]:
logdir = '../my-log-dir/'

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics=['accuracy']
)
  
model_for_pruning.fit(train_generator, 
    steps_per_epoch = nb_train_samples // batch_size, 
    epochs = epochs,
    validation_steps = nb_validation_samples // batch_size, shuffle=True, callbacks = callbacks)

Epoch 1/25
 1/45 [..............................] - ETA: 0s - accuracy: 0.8125 - loss: 0.3441WARNING:tensorflow:From /home/sdmohant/.virtualenvs/python3deep/lib/python3.5/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
45/45 [==============================] - 9s 202ms/step - accuracy: 0.8234 - loss: 0.4140
Epoch 2/25
45/45 [==============================] - 9s 202ms/step - accuracy: 0.8554 - loss: 0.3184
Epoch 3/25
45/45 [==============================] - 9s 201ms/step - accuracy: 0.8331 - loss: 0.3261
Epoch 4/25
45/45 [==============================] - 9s 203ms/step - accuracy: 0.8985 - loss: 0.2573
Epoch 5/25
45/45 [==============================] - 9s 203ms/step - accuracy: 0.8846 - loss: 0.2736
Epoch 6/25
45/45 [==============================] - 9s 201ms/step - accuracy: 0.8957 - loss: 0.2589
Epoch 7/25


#### Export Pruned Model

In [19]:
from tensorflow_model_optimization.sparsity import keras as sparsity

final_model = sparsity.strip_pruning(model_for_pruning)
final_model.summary()

Model: "Custom_CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 96, 96, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 96, 96, 16)        1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        4640      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0

In [20]:
from tensorflow.keras.models import load_model
import numpy as np

# model = tf.keras.models.load_model(final_model)


for i, w in enumerate(final_model.get_weights()):
    print(
        "{} -- Total:{}, Zeros: {:.2f}%".format(
            final_model.weights[i].name, w.size, np.sum(w == 0) / w.size * 100
        )
    )

batch_normalization/gamma:0 -- Total:3, Zeros: 0.00%
batch_normalization/beta:0 -- Total:3, Zeros: 0.00%
batch_normalization/moving_mean:0 -- Total:3, Zeros: 0.00%
batch_normalization/moving_variance:0 -- Total:3, Zeros: 0.00%
conv2d/kernel:0 -- Total:1200, Zeros: 90.00%
conv2d/bias:0 -- Total:16, Zeros: 0.00%
batch_normalization_1/gamma:0 -- Total:16, Zeros: 0.00%
batch_normalization_1/beta:0 -- Total:16, Zeros: 0.00%
batch_normalization_1/moving_mean:0 -- Total:16, Zeros: 0.00%
batch_normalization_1/moving_variance:0 -- Total:16, Zeros: 0.00%
conv2d_1/kernel:0 -- Total:4608, Zeros: 90.00%
conv2d_1/bias:0 -- Total:32, Zeros: 0.00%
conv2d_2/kernel:0 -- Total:9216, Zeros: 90.00%
conv2d_2/bias:0 -- Total:32, Zeros: 0.00%
batch_normalization_2/gamma:0 -- Total:32, Zeros: 0.00%
batch_normalization_2/beta:0 -- Total:32, Zeros: 0.00%
batch_normalization_2/moving_mean:0 -- Total:32, Zeros: 0.00%
batch_normalization_2/moving_variance:0 -- Total:32, Zeros: 0.00%
conv2d_3/kernel:0 -- Total:18432

##### Pruned Model Size and Store

In [21]:
import tempfile
import zipfile

# _, new_pruned_keras_file = tempfile.mkstemp(".h5")

new_pruned_keras_file = "../my-log-dir/saved_model/pruned_model.pb"
print("Saving pruned model to: ", new_pruned_keras_file)
tf.keras.models.save_model(final_model, new_pruned_keras_file, include_optimizer=False)
print(
    "Size of the pruned model: %.2f Mb"
    % (os.path.getsize(new_pruned_keras_file) / float(2 ** 20))
)

Saving pruned model to:  ../my-log-dir/saved_model/pruned_model.pb
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../my-log-dir/saved_model/pruned_model.pb/assets
Size of the pruned model: 0.00 Mb


## Quantize

#### Post training quantization
##### Full integer quantization. 

To 8-bits

In [22]:
keras_model = tf.keras.models.load_model(new_pruned_keras_file)

tflite_fullint_model_file = "../my-log-dir/saved_model/post_fullint_quantized.tflite"

# converter = tf.lite.TFLiteConverter.from_saved_model('../log/saved_model/pruned_model.pb')

converter = tf.lite.TFLiteConverter.from_keras_model(final_model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]



num_calibration_steps = 1
def representative_dataset_gen():
    for _ in range(num_calibration_steps):
        # Get sample input data as a numpy array in a method of your choosing.
        ## Ideally we should do a validation calibration but we are using all of the training data for max acc
#         x =np.concatenate([validation_generator.next()[0] for i in range(validation_generator.__len__())])
        x =np.concatenate([train_generator.next()[0] for i in range(train_generator.__len__())])
        print(x.shape)
        yield [x]
converter.representative_dataset = representative_dataset_gen


converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8

tflite_model = converter.convert()
with open(tflite_fullint_model_file, "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmposl69rti/assets
(911, 96, 96, 3)


### Output Files

`pruned_model.pb` - Pruned model file (file size is not going to reduce but the sparsity is incorporated)

`post_fullint_quantized.tflite` - Integer 8-bit quantized model 


## Quantized Model to OpenMV

Here we show the MicroPython code that uses the `post_fullint_quantized.tflite` file to perform inferencing on the device (OpenMV Cam H7+).


In [23]:
# %load CollisionCam.py
# Dune Collision Camera
# written by EB Goldstein and SD Mohanty
# started 10/2020
# last revision 4/2021

# import what we need
import pyb, sensor, image, time, os, tf, random

#setup LEDs and set into known off state
red_led = pyb.LED(1)
green_led = pyb.LED(2)
red_led.off()
green_led.off()

#red light during setup
red_led.on()

# get sensor set up
sensor.reset()                         # Reset & initialize sensor
sensor.set_pixformat(sensor.RGB565) # Set pixel format to RGB
sensor.set_framesize((sensor.QVGA))      # Set frame size to QVGA (320x240)
sensor.set_windowing((240,240))       # Set window to 240x240
sensor.skip_frames(time=2000)          # Let the camera adjust.

#Load the TFlite model and the labels
net = tf.load('/post_quantized_full_int.tflite', load_to_fb=True)
labels = ['collision', 'no collision']

#turn red off when model is loaded
red_led.off()

#MAIN LOOP

# loop needs to do a few things:
# x-take a picture
# x-record the picture on sd card
# x-do the inference
# x-record the inference in a db w/ rand as name
# x-blink LED for field debugging
# x-delay

while(True):

    #toggle LED for visual indication that script is running
    green_led.toggle()

    #get the image/take the picture
    img = sensor.snapshot()

    #Do the classification and get the object returned by the inference.
    TF_objs = net.classify(img)

    #The object has a output, which is a list of classifcation scores
    #for each of the output channels. this model only has 1 (Collision).
    #So now we extract that float value and print to the serial terminal.

    collision_score = TF_objs[0].output()[0]
    print("Collision = %f" % collision_score)

    #we don;t have an RTC attached now, so we save the images and the
    #collision scores with names according to a random bit stream.

    #generate random bits for stream of names
    rand_label = str(random.getrandbits(30))

    #save image on camera (bit as name)
    img.save("./imgs/" + rand_label + ".jpg")

    #save inference (bit as name, and score) to a file
    with open("./inference.txt", 'a') as file:
        file.write(rand_label + "," + str(collision_score) + "\n")

    #wait some number of milliseconds
    pyb.delay(1000)


ImportError: No module named 'pyb'